In [1]:
import pandas as pd
import numpy as np

In [18]:
from os import chdir
chdir(r"C:\Users\Roozbeh\Sumo\WDC_FewerEdgesMap\All cameras in data 106")

In [19]:
df = pd.read_csv("data.csv",usecols=["CAMERA_ID","DENSITY_VALUE","DL_TIMESTAMP_CITY"],engine='c',infer_datetime_format=False)

In [20]:
df[:2]

,CAMERA_ID,DENSITY_VALUE,DL_TIMESTAMP_CITY
0,114,3701.913725,21-NOV-10 07.17.29.000000000 AM
1,114,3688.501961,21-NOV-10 07.17.44.000000000 AM


In [21]:
len(df)

23632477

In [22]:
df['TIMESTAMP']  = pd.to_datetime(df['DL_TIMESTAMP_CITY'],format="%d-%b-%y %I.%M.%S.000000000 %p")

Let's save to hdf for reduced file size and future loading ease

In [33]:
df.to_hdf("data.hd5",key="df",complib="blosc",complevel=9,mode='w')

In [34]:
df.drop("DL_TIMESTAMP_CITY",axis=1,inplace=True)

In [35]:
df.sort_values(["CAMERA_ID","TIMESTAMP"],inplace=True)

In [36]:
df[:2]

,CAMERA_ID,DENSITY_VALUE,TIMESTAMP
7375546,27,727.027451,2010-11-21 07:16:01
7375547,27,439.537255,2010-11-21 07:16:16


In [37]:
camerasdf = pd.read_csv("cameras.csv",header=0)

In [10]:
#camerasdf.drop(camerasdf.columns[0],axis=1,inplace=True)

In [38]:
len(camerasdf)

106

In [39]:
camerasdf[:5]

,CAMERA_ID,CAMERA_NAME,CAMERA_DESC,ORIENTATION,ZIPCODE,GEO_LAT,GEO_LON
0,114,200144,N Capitol St @ H St,South,20211,38.900343,-77.009251
1,115,200145,Constitution Ave @ 20th St,East,20551,38.891975,-77.045043
2,116,200146,14th St @ Independence Ave,Median,20228,38.887402,-77.031956
3,117,200147,H St @ 14th St,East,20533,38.900149,-77.031760
4,118,200148,14th St @ New York Ave,North,20501,38.899637,-77.031821


In [40]:
camera_groups = df.groupby("CAMERA_ID")

In [41]:
#import matplotlib.pyplot as plt
len(camera_groups)

106

In [42]:
len(df.CAMERA_ID.unique())

106

In [ ]:
temp = list()
for name,group in camera_groups:
    temp.append(name)

In [ ]:
len(temp)

It is already done in sampling but always make sure it is done.

In [56]:
cameras_in_df = camerasdf.merge(df,on="CAMERA_ID")

In [64]:
cameras_in_df[df.columns].to_csv("data_2.csv",index=False)

In [65]:
cameras_in_df[camerasdf.columns].drop_duplicates().to_csv("cameras_2.csv",index=False)

In [43]:
def getFlowFromOcc(x,peakPt = (0.509,38.75),startPt = (0.044,7.15),endPt = (0.80,16)):
    return ( (x-startPt[0]) * (peakPt[1]-startPt[1])/(peakPt[0]-startPt[0]) + startPt[1] 
             if x < peakPt[0] 
             else ( endPt[0]-x ) * (peakPt[1]-endPt[1])/(endPt[0]-peakPt[0]) + endPt[1] )

In [44]:
def getSpeedFromOcc(x,a = 77.45, b = -98.95 , c = 35.9):
    return a * x * x + b * x + c

In [45]:
def get_flow_and_speed_from_group(group):
    #print group
    min_density = group["DENSITY_VALUE"].min()
    group["DENSITY_VALUE_TEMP"] = group["DENSITY_VALUE"]-min_density
    max_density = group["DENSITY_VALUE_TEMP"].max()
    group["DENSITY_VALUE_TEMP"] /= max_density
    group["DENSITY_VALUE_TEMP"] *= 0.80 #it is to scale values matching to occupancy values :-?
    group["FLOW"] = group["DENSITY_VALUE_TEMP"].apply(getFlowFromOcc)
    group["SPEED"] = group["DENSITY_VALUE_TEMP"].apply(getSpeedFromOcc)
    #print group
    return group[["CAMERA_ID","TIMESTAMP","FLOW","SPEED"]]

In [46]:
measurementsdf = camera_groups.apply(get_flow_and_speed_from_group)

In [76]:
measurementsdf[:5]

,CAMERA_ID,TIMESTAMP,FLOW,SPEED
7375546,27,2010-11-21 07:16:01,7.933743,30.643865
7375547,27,2010-11-21 07:16:16,6.345627,32.797544
7375548,27,2010-11-21 07:16:31,6.864088,32.085147
7375549,27,2010-11-21 07:16:46,7.095146,31.770564
7375550,27,2010-11-21 07:17:01,7.012869,31.882377


In [48]:
import datetime

In [49]:
type(measurementsdf.TIMESTAMP[0])

pandas.tslib.Timestamp

In [77]:
measurements_10_1030 = measurementsdf.ix[(measurementsdf.TIMESTAMP  < pd.Timestamp("2010-11-21 10:31:00") ) & 
                                         (measurementsdf.TIMESTAMP >= pd.Timestamp("2010-11-21 10:00:00"))]

In [78]:
measurements_10_1030

,CAMERA_ID,TIMESTAMP,FLOW,SPEED
7375010,27,2010-11-21 10:00:03,10.867133,26.888269
7375011,27,2010-11-21 10:00:18,10.969231,26.762752
7375012,27,2010-11-21 10:00:33,10.686702,27.110944
7375013,27,2010-11-21 10:00:48,8.320168,30.132623
7375014,27,2010-11-21 10:01:03,7.418358,31.333518
7375015,27,2010-11-21 10:01:18,10.493489,27.350603
7375016,27,2010-11-21 10:01:33,11.387696,26.251951
7375017,27,2010-11-21 10:01:48,9.698608,28.349737
7375018,27,2010-11-21 10:02:03,9.313461,28.841473
7375019,27,2010-11-21 10:02:18,10.447737,27.407537


In [79]:
data_30min_avg_20101121_10_1030 = measurements_10_1030.groupby("CAMERA_ID").mean()

In [80]:
data_30min_avg_20101121_10_1030.reset_index(inplace=True)

In [81]:
cameras_in_timespan = cameras_in_df.merge(data_30min_avg_20101121_10_1030,on="CAMERA_ID")

In [82]:
cameras_in_timespan[df.columns].to_csv("data_3.csv",index=False)

In [83]:
camera3_df= cameras_in_timespan[camerasdf.columns].drop_duplicates()
camera3_df.to_csv("cameras_3.csv",index=False)

In [84]:
data_30min_avg_20101121_10_1030.to_csv("data_30min_avg_20101121_10_1030.csv",index=False)

In [62]:
def samplecameras(inp,dist,out=False):
    df = pd.read_csv(inp)
    df["fake_key"] = 1
    cdf = df[["CAMERA_ID","fake_key"]].merge(df[["CAMERA_ID","fake_key"]],on="fake_key",suffixes=["_FROM","_TO"]) # cross product => every possible from_to camera_id
    ddf = pd.read_csv(dist)
    sdf = cdf.merge(ddf,on=["CAMERA_ID_FROM","CAMERA_ID_TO"])  
    if out:
        sdf[ddf.columns].to_csv(out,index=False)
    else:
        return sdf[ddf.columns]

In [85]:
samplecameras("cameras_3.csv","distances.csv","camdist_3.csv")

Run until here and we have flow estimates for 10 to 10:30am 

In [ ]:
measurementsdf.columns = ["Detector","Time","qPKW","vPKW"]

In [ ]:
measurementsdf

In [ ]:
measurementsdf["Time"] = measurementsdf["Time"].astype(np.int64) // 10**9

In [ ]:
measurementsdf["Time"] -= measurementsdf["Time"].min()

In [ ]:
%matplotlib inline
measurementsdf["qPKW"].plot.hist()

In [ ]:
measurementsdf["time_bin"] = (measurementsdf["Time"]/1800).astype(int)
groups_30min = measurementsdf.groupby(["Detector","time_bin"])

In [ ]:
measurementsdf[:9]

In [ ]:
groups_30min_measurement = groups_30min.mean().reset_index()[["Detector","time_bin","qPKW","vPKW"]]

In [ ]:
groups_30min_measurement.columns =["Detector","Time","qPKW","vPKW"]

In [ ]:
groups_30min_measurement[["Detector","qPKW"]].groupby("Detector").mean().reset_index().to_csv("london_100_cameras_30min_data.csv",index=False)

In [ ]:
# count = 0
# def splitIntoLanes(row):
#     num_lanes = camerasdf.loc[camerasdf["CAMERA_ID"]==row["Detector"],"NUM_LANES"]
#     rows = list()
#     for i in range(0,num_lanes):
#         rows.append({"Detector":str(int(row[0]))+"_"+str(i) , "Time":np.int64(row[1]), "qPKW" : (row[2]/int(num_lanes)) , "vPKW" : row[3]})
#     return rows

In [ ]:
# final_measurements = measurementsdf.apply(splitIntoLanes,axis=1)

In [ ]:
# measurementsdf.iloc[129449]

In [ ]:
measurementsdf.to_csv("london_measurements_20_cameras.csv",sep=";",index=False)